In [61]:
%matplotlib inline

In [62]:
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import xarray as xr
import numpy as np
import matplotlib.dates as mdates
import datetime as dt
import time
import sys
import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import math
import imageio
import warnings
import os
warnings.filterwarnings('ignore')

## Functions

In [63]:
def setup_m(ax):
    
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_global()
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.LAND)
    t1 = ax.gridlines(draw_labels=True,alpha=0.4)
    t1.xlabels_top = False
    t1.ylabels_right = False

    return ax

def moving_average(x, w):
    return np.convolve(x, np.ones(w)/w, 'valid')


In [64]:
def vert_avg_winds(u,v):
    # take in wind fields from 300mb to 850mb and average them
    # return u,v vertically averaged (300-850mb) wind fields
    
    # assign empty arrays 
    u_bar = np.zeros(np.shape(u[0,0,:,:]))
    v_bar = np.zeros(np.shape(v[0,0,:,:]))
    
    # assign empty 3d wind arrays 
    lev_bot = int(np.where(u['level']==850)[0])
    lev_top = int(np.where(u['level']==300)[0])
    diff = abs(lev_top-lev_bot)
    wind3du = np.zeros([len(u[0,0,:,0]),len(u[0,0,0,:]),diff])
    wind3dv = np.zeros([len(u[0,0,:,0]),len(u[0,0,0,:]),diff])
    
    # assign wind data to 3d array to easily average
    ii=0
    for i in range(lev_top,lev_bot):
        wind3du[:,:,ii] = u[0,i,:,:]
        wind3dv[:,:,ii] = v[0,i,:,:]
        ii=ii+1
        
    # average over 3rd dim
    u_bar[:,:] = np.mean(wind3du,2)
    v_bar[:,:] = np.mean(wind3dv,2)
    
    return u_bar,v_bar

In [67]:
# plot gif of steering flow
images = []
os.chdir('/home/Tyler.Barbero/tmp/')
filenames = sorted(os.listdir('/home/Tyler.Barbero/tmp/'))
for filename in filenames:
    images.append(imageio.imread(filename))
    imageio.mimsave("/home/Tyler.Barbero/tmp/synoptic_movie.gif", images,duration=0.5)

In [34]:
# # plot gif of TC track
# images = []
# os.chdir('/archive/twb/track_plots/zoom/')
# filenames = sorted(os.listdir('/archive/twb/track_plots/zoom/'))
# for filename in filenames:
#     images.append(imageio.imread(filename))
#     imageio.mimsave("/home/Tyler.Barbero/tmp/TCtracks_0.5sec.gif", images,duration=0.5)

# ################################IFS########################################################################

## Main function (IFS analysis)

In [1]:
root_dir = '/archive/twb/IFS/1_AN0_'
out_dir = '/home/Tyler.Barbero/tmp/'
g = 9.80665

model= 'IFS'
# ini_times = ['20170917','20170918','20170919','20170920','20170921','20170922']
# ini_times = ['20170914','20170915','20170916','20170917','20170918','20170919','20170920','20170921','20170922','20170923']
ini_times = ['20170914','20170915','20170916','20170917','20170918','20170919','20170920','20170921','20170922','20170923','20170924','20170925','20170926','20170927','20170928','20170929','20170930']
ini_hours = ['0','600','1200','1800']

plev = 500
wlev = 850
wnd = "850-300mb avg wind"

########################################################################
########################################################################

for ini_time in ini_times:
    for ini_hour in ini_hours:
        fig = plt.figure(figsize=(11,8))
        fig.patch.set_facecolor('white')
        ax = fig.add_axes([0.1,0.1,0.8,0.8])
        m = setup_m(ax)
        
        fz = root_dir + ini_time + '_' + ini_hour + ".z.nc" #geopotenial height 
        fu = root_dir + ini_time + '_' + ini_hour + ".u.nc" 
        fv = root_dir + ini_time + '_' + ini_hour + ".v.nc"
        dz = xr.open_dataset(fz)
        du = xr.open_dataset(fu)
        dv = xr.open_dataset(fv)
        
        
        # Extract Data 
        h = dz['z']
        lat = du['latitude']
        lon = du['longitude']
        u = du['u']
        v = dv['v']
        
        # get vertically averaged winds 
        u_bar,v_bar = vert_avg_winds(u,v)


        # plot steering flow (sf)
        sf = plt.contour(lon,lat,h[0,int(np.where(dz['level']==plev)[0]),:,:]/g,linewidths=1,levels=np.arange(5800,6100,20), colors='black',transform=ccrs.PlateCarree())
        ax.clabel(sf, fmt='%g',inline=False)
        sf = plt.contour(lon,lat,h[0,int(np.where(dz['level']==plev)[0]),:,:]/g,linewidths=1,levels=np.arange(5500,5800,100), colors='black',transform=ccrs.PlateCarree())
        ax.clabel(sf, fmt='%g',inline=False)
        plt.contour(lon,lat,h[0,int(np.where(dz['level']==plev)[0]),:,:]/g,linewidths=2,levels=[5900],colors='black',transform=ccrs.PlateCarree())
        
        # shade the main steering flow grey
        plt.contourf(lon,lat,h[0,int(np.where(dz['level']==plev)[0]),:,:]/g,levels=[5900,6000],colors='grey',alpha=0.4,transform=ccrs.PlateCarree())
    
        # plot wind vectors at 850 mb
#         plt.barbs(lon[::2],lat[::2],u[0,int(np.where(dz['level']==wlev)[0]),::2,::2],v[0,int(np.where(dz['level']==wlev)[0]),::2,::2]
#                   ,transform=ccrs.PlateCarree(),length=5)
        # plot vertically averaged winds
        plt.barbs(lon[::2],lat[::2],u_bar[::2,::2],v_bar[::2,::2]
                  ,transform=ccrs.PlateCarree(),length=5)
        
        plt.title("500mb Geopotential Height, Model: "+model+" | Init_time: "+ini_time+"_"+ini_hour+"Z",size=12)
        plt.rcParams["font.weight"] = "bold"
        plt.rcParams["axes.labelweight"] = "normal"
        m.set_extent([-101,0,-1,60])
        dz.close();du.close();dv.close()
        if ini_hour=='600':
            ini_hour = '0600'
            fname = 'synoptic_analysis_'+ini_time+"_"+ini_hour+".png"
        elif ini_hour=='0':
            ini_hour = '0000'
            fname = 'synoptic_analysis_'+ini_time+"_"+ini_hour+".png"
        else:
            fname = 'synoptic_analysis_'+ini_time+"_"+ini_hour+".png"
        plt.savefig(out_dir+fname,orientation='landscape',bbox_inches='tight')

NameError: name 'plt' is not defined

# ################################GFS########################################################################

## Main Function (GFS Analysis)

In [ ]:
# initalize arrray
mean_u = np.zeros(np.shape(u[0,0,:,:]))
mean_v = np.zeros(np.shape(u[0,0,:,:]))

# do running mean for each u,v over longitudes
for i in range(0,len(u[0,0,:,0])):
    tmpu = moving_average(u[0,int(np.where(u['lev']==500)[0]),i,:],4)
    tmpv = moving_average(v[0,int(np.where(u['lev']==500)[0]),i,:],4)
    if np.shape(tmpu)!=np.shape(mean_u[0,:]):
        diff = abs(int(np.shape(tmpu)[0])-int(np.shape(mean_u[0,:])[0]))
#         tmpu = np.pad(tmpu,diff,'constant')
#         tmpv = np.pad(tmpv,diff,'constant')
        tmpu = np.concatenate([tmpu, np.zeros(diff)])
        tmpv = np.concatenate([tmpv, np.zeros(diff)])
    mean_u[i,:] = tmpu
    mean_v[i,:] = tmpv

In [ ]:
# Input variables
root_dir = '/archive/twb/GFS/analysis/'
out_dir = '/home/Tyler.Barbero/tmp/'
g = 9.80665

model= 'GFS'

# ini_times = ['20170917','20170918','20170919','20170920','20170921','20170922']
ini_times = ['20170916','20170917']
ini_hours = ['00','12']

########################################################################
########################################################################

for ini_time in ini_times:
    for ini_hour in ini_hours:
        fig = plt.figure(figsize=(11,8))
        fig.patch.set_facecolor('white')
        ax = fig.add_axes([0.1,0.1,0.8,0.8])
        m = setup_m(ax)
        
        # we want high resolution T1534 data for geopotential height
        fz = root_dir + ini_time + '_' + ini_hour + "Z00_T1534_h.nc"
        fu = root_dir + ini_time + '_' + ini_hour + "Z00_T1534_u.nc" 
        fv = root_dir + ini_time + '_' + ini_hour + "Z00_T1534_v.nc"
        dz = xr.open_dataset(fz)
        du = xr.open_dataset(fu)
        dv = xr.open_dataset(fv)
        
        
        # Extract Data 
        h = dz['hgt']
        lat = du['lat']
        lon = du['lon']
        u = du['uwind']
        v = dv['vwind']
        

        # plot all steering flow (sf) 
        sf = plt.contour(lon,lat,h[0,int(np.where(dz['lev']==500)[0]),:,:],linewidths=1,levels=np.arange(5500,5800,100), colors='black',transform=ccrs.PlateCarree())
        ax.clabel(sf, fmt='%g',inline_spacing=1)
        sf = plt.contour(lon,lat,h[0,int(np.where(dz['lev']==500)[0]),:,:],linewidths=2,levels=np.arange(5800,5950,20), colors='black',transform=ccrs.PlateCarree())
        ax.clabel(sf, fmt='%g',inline_spacing=1)

        # shade the main steering flow grey
        plt.contourf(lon,lat,h[0,int(np.where(dz['lev']==500)[0]),:,:],levels=[5900,6000],colors='grey',alpha=0.4,transform=ccrs.PlateCarree())
#         plt.contourf(lon,lat,h[0,int(np.where(dz['lev']==500)[0]),:,:],colors='grey',alpha=0.4,transform=ccrs.PlateCarree())
    
        # plot wind vectors at 850 mb
#         plt.barbs(lon[::20],lat[::20],u[0,int(np.where(dz['lev']==850)[0]),::20,::20],v[0,int(np.where(dz['lev']==850)[0]),::20,::20]
#                   ,transform=ccrs.PlateCarree(),length=5)
        plt.barbs(lon[::24],lat[::24],mean_u[::24,::24],mean_v[::24,::24],transform=ccrs.PlateCarree(),length=5)

        
        plt.title("500mb Geopotential Height Steering Flow, 850mb wind,    model: "+model+"     Ini_time: "+ini_time+"_"+ini_hour+"Z",size=15)
        m.set_extent([-101,1,9,61])
        dz.close();du.close();dv.close()
#         fname = 'synoptic_analysis_'+ini_time+"_"+ini_hour+".png"
#         plt.savefig(out_dir+fname,orientation='landscape',bbox_inches='tight')

In [ ]:
a = moving_average(u[0,int(np.where(u['lev']==500)[0]),52,:],4)
b = u[0,int(np.where(u['lev']==500)[0]),0,:]
fig = plt.figure(figsize=(11,8))
plt.plot(np.arange(0,len(a)),a,label='running mean')
plt.plot(np.arange(0,3072),b,label='orig')
plt.legend()

fig = plt.figure(figsize=(11,8))
a = np.random.randint(1,100,100)
b = moving_average(a,5)
plt.plot(np.arange(0,len(a)),a,label='a - orig data')
plt.plot(np.arange(0,len(b)),b,label='b - running mean')
plt.legend()
